## Introducción

Este notebook analiza un dataset relacionado con niveles de burnout mediante técnicas de clustering.
se ha desarrollado un algoritmo que permite revisar automáticamente
los valores máximos y mínimos dentro de cada clúster generado, facilitando la interpretación
de los grupos y la comparación entre ellos.

Se observo que los valores maximos y minimos no coincidian exactamente con cada nivel low,medium y high en el burnout, por lo que se hizo una reclasificacion y permitio entender patrones que van alla de una etiqueta preestablecida.

A través de este enfoque, se identifican patrones de comportamiento asociados a distintos niveles
de burnout, combinando análisis exploratorio, segmentación de datos y evaluación de resultados
basada en métricas estadísticas.


In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('work_from_home_burnout_dataset.csv')

## Exploración inicial de datos

Se realiza una inspección inicial del dataset para comprender su estructura,
tipos de variables y posibles valores faltantes, estableciendo una base sólida
para el análisis posterior.


In [3]:
df.head()

,user_id,day_type,work_hours,screen_time_hours,meetings_count,breaks_taken,after_hours_work,sleep_hours,task_completion_rate,burnout_score,burnout_risk
0,1,Weekday,9.59,11.86,4,2,0,7.55,91.2,19.17,Low
1,1,Weekend,7.38,10.33,4,1,0,6.69,82.0,29.70,Low
2,1,Weekend,6.31,8.92,1,2,0,8.87,80.6,32.93,Low
3,1,Weekday,8.34,10.70,4,1,1,8.13,70.0,45.47,Low
4,1,Weekend,6.97,9.83,1,2,0,5.85,67.1,51.61,Low


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   user_id               1800 non-null   int64  
 1   day_type              1800 non-null   object 
 2   work_hours            1800 non-null   float64
 3   screen_time_hours     1800 non-null   float64
 4   meetings_count        1800 non-null   int64  
 5   breaks_taken          1800 non-null   int64  
 6   after_hours_work      1800 non-null   int64  
 7   sleep_hours           1800 non-null   float64
 8   task_completion_rate  1800 non-null   float64
 9   burnout_score         1800 non-null   float64
 10  burnout_risk          1800 non-null   object 
dtypes: float64(5), int64(4), object(2)
memory usage: 154.8+ KB


In [5]:
df['burnout_risk'].unique()

array(['Low', 'Medium', 'High'], dtype=object)

## Análisis descriptivo

Se calculan estadísticas básicas del dataset para entender la distribución de las variables
y detectar posibles valores atípicos que puedan influir en el proceso de clustering.


In [6]:
print(df.loc[df['burnout_risk'] == 'Low', 'burnout_score'].min())
print(df.loc[df['burnout_risk']=='Low', 'burnout_score'].max())

2.5
69.92


In [7]:
print(df.loc[df['burnout_risk']=='Medium','burnout_score'].min())
print(df.loc[df['burnout_risk']=='Medium','burnout_score'].max())

70.01
109.99


In [8]:
print(df.loc[df['burnout_risk']=='High','burnout_score'].min())
print(df.loc[df['burnout_risk']=='High','burnout_score'].max())

110.22
143.92


## Preprocesamiento de datos

Los datos son preparados mediante la selección de variables relevantes y el escalado de características.
Este paso es fundamental para asegurar que el algoritmo de clustering funcione correctamente
y no se vea sesgado por diferencias de magnitud entre variables.


In [9]:
from sklearn.cluster import KMeans

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   user_id               1800 non-null   int64  
 1   day_type              1800 non-null   object 
 2   work_hours            1800 non-null   float64
 3   screen_time_hours     1800 non-null   float64
 4   meetings_count        1800 non-null   int64  
 5   breaks_taken          1800 non-null   int64  
 6   after_hours_work      1800 non-null   int64  
 7   sleep_hours           1800 non-null   float64
 8   task_completion_rate  1800 non-null   float64
 9   burnout_score         1800 non-null   float64
 10  burnout_risk          1800 non-null   object 
dtypes: float64(5), int64(4), object(2)
memory usage: 154.8+ KB


In [11]:
X = df.drop(['user_id','day_type','burnout_risk'], axis=1)

## Clustering

Se aplican algoritmos de clustering para segmentar los datos en distintos grupos.
El objetivo es identificar perfiles diferenciados de burnout en función
de los patrones presentes en las variables analizadas.


In [12]:
kmeans=KMeans(n_clusters=3,random_state=0)
kmeans.fit(X)

KMeans(n_clusters=3, random_state=0)

In [13]:
grupos=kmeans.labels_

In [14]:
df['groups']=grupos

In [15]:
df.head()

,user_id,day_type,work_hours,screen_time_hours,meetings_count,breaks_taken,after_hours_work,sleep_hours,task_completion_rate,burnout_score,burnout_risk,groups
0,1,Weekday,9.59,11.86,4,2,0,7.55,91.2,19.17,Low,1
1,1,Weekend,7.38,10.33,4,1,0,6.69,82.0,29.70,Low,1
2,1,Weekend,6.31,8.92,1,2,0,8.87,80.6,32.93,Low,1
3,1,Weekday,8.34,10.70,4,1,1,8.13,70.0,45.47,Low,0
4,1,Weekend,6.97,9.83,1,2,0,5.85,67.1,51.61,Low,0


In [16]:
df.loc[df['burnout_risk'] == 'Low', 'groups'].value_counts()

groups
1    811
0    671
2     45
Name: count, dtype: int64

## Análisis de clústeres

Se utiliza el algoritmo desarrollado para analizar cada clúster,
obteniendo los valores máximos y mínimos de las variables.
Este análisis permite interpretar las características extremas de cada grupo
y compararlos de manera objetiva.


In [17]:
cols=['Low','Medium','High']
j=-1
for i in cols:
    j=j+1
    print(i)
    print(df.loc[df['burnout_risk']==i,'groups'].value_counts())
    print(f' The value max is found in {df.loc[df['groups']==j,'burnout_risk'].max()}')
    print(f' The value min is found in {df.loc[df['groups']==j,'burnout_risk'].min()}')

Low
groups
1    811
0    671
2     45
Name: count, dtype: int64
 The value max is found in Low
 The value min is found in Low
Medium
groups
2    253
Name: count, dtype: int64
 The value max is found in Low
 The value min is found in Low
High
groups
2    20
Name: count, dtype: int64
 The value max is found in Medium
 The value min is found in High


# Dado que las etiquetas Low, Medium y High no capturan los extremos reales del burnout y presentan alta heterogeneidad, se realizó una reclasificación basada en clusters, identificando perfiles de bajo riesgo estable, bajo riesgo aparente y riesgo elevado.

In [18]:
def new_burnout_class(group):
    if group == 0:
        return 'Bajo estable'
    elif group == 1:
        return 'Bajo aparente'
    elif group == 2:
        return 'Riesgo elevado'

df['burnout_new'] = df['groups'].apply(new_burnout_class)
